In [2]:
import pandas as pd
import altair as alt
import numpy as np

c:\Users\merii\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df = pd.read_excel('../data/Marina_ciclo2.xlsx', engine = 'openpyxl' )

In [7]:
df['Max_Last_Six'] = df['Temperatura'].rolling(window=6, min_periods=6).apply(lambda x: x[:-1].max())

mask = (df['Temperatura'] >= df['Max_Last_Six']) & ~(df['Temperatura'].shift(1) >= df['Max_Last_Six'].shift(1))

# Find the index of the first occurrence
first_occurrence_index = mask.idxmax()

# Set all values to False except for the first occurrence
df['First_Over_Max_Last_Six'] = False
df.loc[first_occurrence_index, 'First_Over_Max_Last_Six'] = True

# Get that temperature
baseline = df[df.First_Over_Max_Last_Six == True].Max_Last_Six.iloc[0]
if np.isnan(baseline):
    baseline=40

df['baseline'] = baseline


# Check the next three temperatures are higher than that value
df['higher_baseline']=df.apply(lambda x: ((x.Temperatura >= x.baseline+0.1)), axis=1)

df['days_higher_than_baseline'] = df['higher_baseline'].rolling(window=3, min_periods = 1).sum()

# confirm ovulation
df['ovulation_confirmed'] = False
mask_ovulation = ((df.ovulation_confirmed == False) & (df.days_higher_than_baseline==3))
first_occurrence_ovulation = mask_ovulation.idxmax()
df.loc[first_occurrence_ovulation::, 'ovulation_confirmed'] = True

if df.loc[0, 'ovulation_confirmed']:
    df.loc[0, 'ovulation_confirmed'] = False

In [12]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Plot
line = alt.Chart(df).mark_line().encode(
    x='Fecha:T',
    y=alt.Y('Temperatura:Q', scale=alt.Scale(domain=[36, 37]))
)

points = alt.Chart(df).mark_point().encode(
    x='Fecha:T',
    y='Temperatura:Q',
    tooltip=['Fecha:T', 'Temperatura:Q']
)

baseline = alt.Chart(df).mark_line().encode(
    x='Fecha:T',
    y=alt.Y('baseline:Q', scale=alt.Scale(domain=[36, 37]))
)

points_ovulation = alt.Chart(df[df['ovulation_confirmed']]).mark_point(color='green', filled = True, size = 100).encode(
    x='Fecha:T',
    y='Temperatura:Q',
    tooltip=['Fecha:T', 'ovulation_confirmed']
)

chart = (line + points+baseline+points_ovulation).properties(
    title='Temperatura vs Fecha',
    width=600,
    height=300
)

chart.configure_axis(
    labelAngle=45
)

alt.LayerChart(...)